In [15]:
import pandas as pd
import numpy as np
np.random.seed(5)

In [16]:
global  TEMPLATE_NEGATIVE
global TEMPLATE_POSITIVE

In [17]:
RACE=['White','Asian','Black']
NATIONALITY=['Finnish','Taiwanese','Dutch','Chinese']
#RElIGION=['Christian','Buddhist','Muslim','Hindu','Jewish']
#SEXUAL_ORIENTAION=['Heterosexual','Homosexual','Bisexual']

TEMPLATE_NEGATIVE=['I am <IDENTITY>',
'I am a <IDENTITY> person',
'<IDENTITY> people are just like everyone else']

TEMPLATE_POSITIVE=[
'I hate all <IDENTITY>',
'I am a <IDENTITY> person and I hate your guts and think you suck',
'<IDENTITY> people are gross']
# Might need to translate to different langugage

In [18]:
def Session(idenList1,idenList2,diff=False):
    
    if diff:
        index=np.random.permutation(len(idenList1))
        iden1=idenList1[index[0]]
        iden2=idenList2[index[1]]
    else:
        iden1=idenList1[np.random.randint(len(idenList1))]
        iden2=idenList2[np.random.randint(len(idenList2))]
    
    indexNegative=np.random.permutation(len(TEMPLATE_NEGATIVE))
    indexPositive=np.random.permutation(len(TEMPLATE_POSITIVE))
    
    texts=[]
    texts+=[TEMPLATE_POSITIVE[indexPositive[0]].replace("<IDENTITY>", iden1)]
    texts+=[TEMPLATE_POSITIVE[indexPositive[1]].replace("<IDENTITY>", iden2)]
    texts+=[TEMPLATE_NEGATIVE[indexNegative[0]].replace("<IDENTITY>", iden1)]
    texts+=[TEMPLATE_NEGATIVE[indexNegative[1]].replace("<IDENTITY>", iden2)]
    
    labels=[True,True,False,False]
    modelA=[True,True,True,False]
    modelB=[True,False,False,False]
    eqModelA='True Positive Rate Equilibrium'
    eqModelB='False Positive Rate Equilibrium'

    # Shuffle model A and model B
    if np.random.permutation(2)[0]:
        eqModelA='False Positive Rate Equilibrium'
        eqModelB='True Positive Rate Equilibrium'
        modelA=[True,False,False,False]
        modelB=[True,True,True,False]
        
        
    sessionnDF=pd.DataFrame(texts,columns=['Comment'])
    sessionnDF['True OutComes']=labels
    sessionnDF['ModelA_Prediction']=modelA
    sessionnDF['ModelB_Prediction']=modelB
    sessionnDF=sessionnDF.sample(frac=1)

    return sessionnDF,eqModelA,eqModelB

In [19]:
def Generator(properties):
    Path=[]
    EqA=[]
    EqB=[]
    for i in range(len(properties)):
        
        # Generate csv file for the same identity
        listB=properties.copy()
        listB.remove(listB[i])
        
        sessionnDF,eqModelA,eqModelB = Session(properties[i:i+1],listB,diff=False)
        path='csv/'+properties[i]+'.csv'#+'_Same.csv'
        sessionnDF.to_csv(path,index=False)       
        Path+=[path]
        EqA+=[eqModelA]
        EqB+=[eqModelB]
        '''
        # Generate csv file for different identity
        sessionnDF,eqModelA,eqModelB = Session(listB,listB,diff=True)
        path='csv/'+properties[i]+'_Diff.csv'
        sessionnDF.to_csv(path,index=False)                                                  
        Path+=[path]
        EqA+=[eqModelA]
        EqB+=[eqModelB]     
        '''
    csvfileProperties=pd.DataFrame(Path,columns=['Path'])
    csvfileProperties['ModelA']=EqA
    csvfileProperties['ModelB']=EqB
    return csvfileProperties

In [20]:
prop_RACE=Generator(RACE)
prop_NATIONALITY=Generator(NATIONALITY)

In [21]:
CsvfilePropertiespd=pd.concat([prop_RACE, prop_NATIONALITY], ignore_index=True)
CsvfilePropertiespd.to_csv('Csvfile_Properties.csv',index=False)